In [358]:
df.head(3)

,_id,Data Extract Date,Overdose Death Year,Overdose Death Drug Class,Overdose Death Geography Level,Overdose Death FIPS,Overdose Death Geography Name,Overdose Death Health District,Overdose Death Count,"Overdose Death Rate per 100,000 Residents"
0,1,6/2/2025,2021,Any Opioids,Locality,51175,Southampton County,Western Tidewater,2,11.3
1,2,6/2/2025,2021,Benzodiazepine,Locality,51147,Prince Edward County,Piedmont,0,0
2,3,6/2/2025,2020,Any Opioids,Locality,51690,Martinsville City,West Piedmont,8,64.8


In [4]:
import sklearn
import sklearn.manifold as manifold
import umap
import kmapper as km
import numpy as np
import matplotlib.pyplot as plt
from kmapper.jupyter import display
import pandas as pd

In [ ]:
df = pd.read_csv('/Users/joshturner/Desktop/DrugOverdoseTDAVA/Data/OverdoseDeathsByRegion.csv')
# Dataset: https://data.virginia.gov/dataset/vdh-pud-overdose-deaths-by-year-and-geography/resource/a3a62450-44f6-4db2-b400-318c57674b11

In [15]:
df.isna().sum()

_id                                            0
Data Extract Date                              0
Overdose Death Year                            0
Overdose Death Drug Class                      0
Overdose Death Geography Level                 0
Overdose Death FIPS                            0
Overdose Death Geography Name                  0
Overdose Death Health District               154
Overdose Death Count                           0
Overdose Death Rate per 100,000 Residents     66
dtype: int64

# Grabs the longitude and latitude for each city

In [24]:
counties = df["Overdose Death Geography Name"].unique()

In [26]:
import pandas as pd
from geopy.geocoders import Nominatim
from time import sleep

# Initialize geolocator
geolocator = Nominatim(user_agent="va_county_geocoder")

# Create empty lists to store results
latitudes = []
longitudes = []

# Loop over each county
for county in counties:
    location = None
    try:
        # Add 'Virginia, USA' to narrow it down
        location = geolocator.geocode(f"{county}, Virginia, USA")
        if location:
            latitudes.append(location.latitude)
            longitudes.append(location.longitude)
        else:
            latitudes.append(None)
            longitudes.append(None)
    except Exception as e:
        print(f"Error geocoding {county}: {e}")
        latitudes.append(None)
        longitudes.append(None)
    
    # Be polite: Nominatim recommends 1-second delay
    sleep(1)

# Combine into a DataFrame
df_geo = pd.DataFrame({
    "County": counties,
    "Latitude": latitudes,
    "Longitude": longitudes
})

df_geo

Error geocoding Rockbridge County: Service timed out
Error geocoding Lexington City: Service timed out


,County,Latitude,Longitude
0,Southampton County,36.695938,-77.158600
1,Prince Edward County,37.229329,-78.462857
2,Martinsville City,36.690971,-79.875039
3,Buchanan County,37.260045,-82.032033
4,Warren County,38.913191,-78.209666
...,...,...,...
130,Lexington City,NaN,NaN
131,Mecklenburg County,36.679848,-78.314746
132,Accomack County,37.742221,-75.674354
133,Charlottesville City,38.029899,-78.477030


In [ ]:
# Fill in the missing longitude and latitude coordinates (luckily for this dataset there were only two)
missing_coords = {
    "Rockbridge County": (37.7776, -79.4308),
    "Lexington City": (37.7840, -79.4428)
}

# Fill in missing manually
for county, (lat, lon) in missing_coords.items():
    df_geo.loc[df_geo["County"] == county, "Latitude"] = lat
    df_geo.loc[df_geo["County"] == county, "Longitude"] = lon


# Merge both datasets

In [30]:
# Merge based on matching county/city names
df_combined = pd.merge(
    df,
    df_geo,
    left_on="Overdose Death Geography Name",
    right_on="County",
    how="left"  # keep all rows in df, even if there's no match in df_geo
)


In [31]:
df_combined.head(10)

,_id,Data Extract Date,Overdose Death Year,Overdose Death Drug Class,Overdose Death Geography Level,Overdose Death FIPS,Overdose Death Geography Name,Overdose Death Health District,Overdose Death Count,"Overdose Death Rate per 100,000 Residents",County,Latitude,Longitude
0,1,6/2/2025,2021,Any Opioids,Locality,51175,Southampton County,Western Tidewater,2,11.3,Southampton County,36.695938,-77.158600
1,2,6/2/2025,2021,Benzodiazepine,Locality,51147,Prince Edward County,Piedmont,0,0,Prince Edward County,37.229329,-78.462857
2,3,6/2/2025,2020,Any Opioids,Locality,51690,Martinsville City,West Piedmont,8,64.8,Martinsville City,36.690971,-79.875039
3,4,6/2/2025,2025†,Natural and Semi-Synthetic Opioids,Locality,51027,Buchanan County,Cumberland Plateau,0,0,Buchanan County,37.260045,-82.032033
4,5,6/2/2025,2019,Benzodiazepine,Locality,51187,Warren County,Lord Fairfax,5,12.4,Warren County,38.913191,-78.209666
5,6,6/2/2025,2022,Psychostimulant,Locality,51735,Poquoson City,Peninsula,0,0,Poquoson City,37.121813,-76.396506
6,7,6/2/2025,2024^,Methadone,Locality,51053,Dinwiddie County,Crater,0,0,Dinwiddie County,37.071282,-77.648071
7,8,6/2/2025,2019,Natural and Semi-Synthetic Opioids,Locality,51113,Madison County,Rappahannock/Rapidan,2,15.1,Madison County,38.411462,-78.284663
8,9,6/2/2025,2024^,Heroin,Locality,51035,Carroll County,Mount Rogers,0,0,Carroll County,36.736582,-80.743979
9,10,6/2/2025,2023,Any Opioids,Locality,51057,Essex County,Three Rivers,1,9.1,Essex County,37.914059,-76.932638


In [32]:
df_combined.to_csv('/Users/joshturner/Desktop/DrugOverdoseTDAVA/Data/VDHPUDOverdoseDeathsByYearAndGeographyLongLat.csv')


In [47]:
df_combined.columns

Index(['_id', 'Data Extract Date', 'Overdose Death Year',
       'Overdose Death Drug Class', 'Overdose Death Geography Level',
       'Overdose Death FIPS', 'Overdose Death Geography Name',
       'Overdose Death Health District', 'Overdose Death Count',
       'Overdose Death Rate per 100,000 Residents', 'County', 'Latitude',
       'Longitude'],
      dtype='object')

# Prep for Mapper

In [293]:
df.nunique()

_id                                          20702
Data Extract Date                                1
Overdose Death Year                              7
Overdose Death Drug Class                       11
Overdose Death Geography Level                   2
Overdose Death FIPS                            135
Overdose Death Geography Name                  135
Overdose Death Health District                  36
Overdose Death Count                           196
Overdose Death Rate per 100,000 Residents      586
dtype: int64

In [296]:
df[df["Overdose Death Geography Name"] == df["Overdose Death Geography Name"].unique().tolist()[0]]

,_id,Data Extract Date,Overdose Death Year,Overdose Death Drug Class,Overdose Death Geography Level,Overdose Death FIPS,Overdose Death Geography Name,Overdose Death Health District,Overdose Death Count,"Overdose Death Rate per 100,000 Residents"
0,1,6/2/2025,2021,Any Opioids,Locality,51175,Southampton County,Western Tidewater,2,11.3
110,111,6/2/2025,2019,Cocaine,Locality,51175,Southampton County,Western Tidewater,0,0
142,143,6/2/2025,2023,Fentanyl and Other Synthetic Opioids,Locality,51175,Southampton County,Western Tidewater,1,5.7
516,517,6/2/2025,2019,Fentanyl and Other Synthetic Opioids,Locality,51175,Southampton County,Western Tidewater,1,5.7
613,614,6/2/2025,2020,Any Opioids,Locality,51175,Southampton County,Western Tidewater,4,22.7
...,...,...,...,...,...,...,...,...,...,...
19712,19713,6/2/2025,2019,Methadone,Locality,51175,Southampton County,Western Tidewater,0,0
19750,19751,6/2/2025,2022,Natural and Semi-Synthetic Opioids,Locality,51175,Southampton County,Western Tidewater,1,5.7
20117,20118,6/2/2025,2025†,Methadone,Locality,51175,Southampton County,Western Tidewater,0,0
20177,20178,6/2/2025,2021,Natural and Semi-Synthetic Opioids,Locality,51175,Southampton County,Western Tidewater,1,5.7


In [321]:
features = ['County','Overdose Death Drug Class','Overdose Death Year','Overdose Death Count', 'Latitude', 'Longitude']

mapper_data = df_combined[features]

In [322]:
# Keep only digits in the Year column
mapper_data['Overdose Death Year'] = mapper_data['Overdose Death Year'].str.extract('(\d{4})')  # Extracts 4-digit year
mapper_data['Overdose Death Year'] = pd.to_numeric(mapper_data['Overdose Death Year'], errors='coerce')  # Convert to numeric (int or NaN)
# Keep only digits in the Year column
# mapper_data['Overdose Death Count'] = mapper_data['Overdose Death Year'].str.extract('(\d{4})')  # Extracts 4-digit year
mapper_data['Overdose Death Count'] = pd.to_numeric(mapper_data['Overdose Death Count'], errors='coerce')  # Convert to numeric (int or NaN)


<>:2: SyntaxWarning:

invalid escape sequence '\d'

<>:2: SyntaxWarning:

invalid escape sequence '\d'

/var/folders/0l/vj8n1smx36d4lztxy5t6_zph0000gq/T/ipykernel_72754/1971826016.py:2: SyntaxWarning:

invalid escape sequence '\d'

/var/folders/0l/vj8n1smx36d4lztxy5t6_zph0000gq/T/ipykernel_72754/1971826016.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/0l/vj8n1smx36d4lztxy5t6_zph0000gq/T/ipykernel_72754/1971826016.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/0l/vj8n1smx

In [323]:
mapper_data.dropna(inplace=True)

/var/folders/0l/vj8n1smx36d4lztxy5t6_zph0000gq/T/ipykernel_72754/4074578487.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [324]:
mapper_data.shape

(20384, 6)

In [325]:
X["Overdose Death Count"].describe()

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [326]:
mapper_data[mapper_data["County"] == mapper_data["County"].unique().tolist()[0]]

,County,Overdose Death Drug Class,Overdose Death Year,Overdose Death Count,Latitude,Longitude
0,Southampton County,Any Opioids,2021,2.0,36.695938,-77.1586
110,Southampton County,Cocaine,2019,0.0,36.695938,-77.1586
142,Southampton County,Fentanyl and Other Synthetic Opioids,2023,1.0,36.695938,-77.1586
516,Southampton County,Fentanyl and Other Synthetic Opioids,2019,1.0,36.695938,-77.1586
613,Southampton County,Any Opioids,2020,4.0,36.695938,-77.1586
...,...,...,...,...,...,...
19712,Southampton County,Methadone,2019,0.0,36.695938,-77.1586
19750,Southampton County,Natural and Semi-Synthetic Opioids,2022,1.0,36.695938,-77.1586
20117,Southampton County,Methadone,2025,0.0,36.695938,-77.1586
20177,Southampton County,Natural and Semi-Synthetic Opioids,2021,1.0,36.695938,-77.1586


In [327]:
df.iloc[[2,10353]]

,_id,Data Extract Date,Overdose Death Year,Overdose Death Drug Class,Overdose Death Geography Level,Overdose Death FIPS,Overdose Death Geography Name,Overdose Death Health District,Overdose Death Count,"Overdose Death Rate per 100,000 Residents"
2,3,6/2/2025,2020,Any Opioids,Locality,51690,Martinsville City,West Piedmont,8,64.8
10353,10354,6/2/2025,2020,Any Opioids,Locality,51690,Martinsville City,West Piedmont,8,64.8


In [328]:
mapper_data = mapper_data[mapper_data['County'] != 'Virginia']

In [329]:
mapper_data = mapper_data[mapper_data['Overdose Death Year'] == 2020]

In [330]:
mapper_data = mapper_data[mapper_data['Overdose Death Drug Class'] == "Any Opioids"]

In [331]:
mapper_data = mapper_data.drop_duplicates()

In [332]:
print("Total rows:", len(mapper_data))
print("Unique rows:", len(mapper_data.drop_duplicates()))


Total rows: 134
Unique rows: 134


In [333]:
mapper_data[mapper_data.duplicated(keep=False)]


,County,Overdose Death Drug Class,Overdose Death Year,Overdose Death Count,Latitude,Longitude


# Drug Overdoses By Count (Begin)

In [ ]:
import kmapper as km
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import sklearn
import pandas as pd
from sklearn import ensemble
from kmapper.plotlyviz import *
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from ipywidgets import (HBox, VBox)

# Step 1: Select and standardize your features
map_features = ['Latitude', 'Longitude','Overdose Death Count']


# Calculate the mean
mean_value = mapper_data['Overdose Death Count'].mean()
# Create a new binary column
mapper_data['Overdose Binary'] = (mapper_data['Overdose Death Count'] > mean_value).astype(int)
# Optional: label as 'low'/'high' instead of 0/1
mapper_data['Overdose Level'] = mapper_data['Overdose Binary'].map({0: 'low', 1: 'high'})

X = np.array(mapper_data[map_features])
y = np.array(mapper_data['Overdose Level'])
# Isolation Forest for lens1
model = ensemble.IsolationForest(random_state=1729)
model.fit(X)
lens1 = model.decision_function(X).reshape((X.shape[0],1))



#X_scaled = StandardScaler().fit_transform(X)

# Step 2: Define a lens function (1D)
# lens = mapper_data['Overdose Death Year'].values.reshape(-1, 1)  # <- Optional but useful as a lens

# Optional: Standardize the lens too (depends on whether year differences are meaningful)
# from sklearn.preprocessing import MinMaxScaler
# lens_scaled = MinMaxScaler().fit_transform(lens)

# Step 3: Initialize Mapper object
mapper = km.KeplerMapper(verbose=0)
# l2 norm for lens2
lens2 = mapper.fit_transform(X,projection="l2norm")
lens = np.c_[lens1,lens2]
# Step 4: Fit the data using the lens
scomplex = mapper.map(
    lens,
    # lens_scaled,              # <- this is the lens
    X,                 # <- this is the data
    cover=km.Cover(n_cubes=5, perc_overlap=0.1),
   #clusterer=DBSCAN(eps=0.5, min_samples=3))
   clusterer=sklearn.cluster.KMeans(n_clusters=2,
                                     random_state=3471))

#tooltips = np.array(mapper_data["County"].tolist()) # Format counties)

# Step 5: Visualize
#mapper.visualize(
#    graph,
#    path_html="mapper_overdose_2lens.html",
#    title="TDA Mapper: Overdose Patterns by County and Year",
#    custom_tooltips=tooltips  # Optional, for hover info
#)


In [351]:
global_mean_deaths = mapper_data['Overdose Death Count'].mean()


In [352]:
global_mean_deaths

np.float64(13.992537313432836)

In [353]:
pl_brewer = [[0.0, '#006837'],
             [0.1, '#1a9850'],
             [0.2, '#66bd63'],
             [0.3, '#a6d96a'],
             [0.4, '#d9ef8b'],
             [0.5, '#ffffbf'],
             [0.6, '#fee08b'],
             [0.7, '#fdae61'],
             [0.8, '#f46d43'],
             [0.9, '#d73027'],
             [1.0, '#a50026']]


In [354]:
custom_tooltips = []

for i, row in mapper_data.iterrows():
    county = row["County"]
    death = row["Overdose Death Count"]
    level = "High" if death > global_mean_deaths else "Low"
    
    tooltip_text = (
        f"<b>Death Count:</b> {death}<br>"
        f"<b>Level:</b> {level}<br>"
        f"<b>County:</b> {county}"
    )
    
    custom_tooltips.append(tooltip_text)
custom_tooltips = np.array(custom_tooltips)

In [355]:
color_values = mapper_data['Overdose Death Count'].values  # per data point
#my_colorscale = 'plasma'
import plotly.colors as pc

my_colorscale = pc.sequential.Plasma  # ← this is valid

kmgraph, mapper_summary, colorf_distribution = get_mapper_graph(
    scomplex,
    color_values=color_values,
    color_function_name="Average Overdose Deaths",
    colorscale=pl_brewer,
    custom_tooltips=custom_tooltips  # ← this is key
)


In [357]:
mapper.visualize(
    scomplex,
    path_html="mapper_opioid_overdose_2020.html",
    title="Overdose Deaths By Opioids Mapper 2020",
    color_values=color_values,
    color_function_name="Overdose Death Count",
    colorscale=pl_brewer,
    custom_tooltips=custom_tooltips
)


'<!DOCTYPE html>\n<html>\n\n<head>\n  <meta charset="utf-8">\n  <meta name="generator" content="KeplerMapper">\n  <title>Overdose Deaths By Opioids Mapper 2020 | KeplerMapper</title>\n\n  <link rel="icon" type="image/png" href="http://i.imgur.com/axOG6GJ.jpg" />\n\n  <link href=\'https://fonts.googleapis.com/css?family=Roboto+Mono:700,300\' rel=\'stylesheet\' type=\'text/css\'>\n  <style>* {\n  margin: 0;\n  padding: 0;\n}\n\nhtml, body {\n  height: 100%;\n}\n\nbody {\n  font-family: "Roboto Mono", "Helvetica", sans-serif;\n  font-size: 14px;\n}\n\n#logo {\n  width:  85px;\n  height: 85px;\n}\n\n#display {\n  color: #95A5A6;\n  background: #212121;\n}\n\n#header {\n  background: #111111;\n}\n\n#print {\n  color: #000;\n  background: #FFF;\n}\n\nh1 {\n  font-size: 21px;\n  font-weight: 300;\n  font-weight: 300;\n}\n\nh2 {\n  font-size: 18px;\n  padding-bottom: 20px;\n  font-weight: 300;\n}\n\nh3 {\n  font-size: 14px;\n  font-weight: 700;\n  text-transform: uppercase;\n}\n\nh4 {\n  font-

# Drug Overdoses By Count (End)

In [342]:
mapper_data.shape

(134, 8)

In [341]:
import pandas as pd

# Step 1: Compute global average
global_mean_deaths = mapper_data['Overdose Death Count'].mean()

# Step 2: Build summary per node
node_summaries = []

for node_id, member_ids in scomplex['nodes'].items():
    counties = mapper_data.iloc[member_ids]['County'].tolist()
    avg_deaths = mapper_data.iloc[member_ids]['Overdose Death Count'].mean()
    level = "High" if avg_deaths > global_mean_deaths else "Low"

    summary = {
        "Node ID": node_id,
        "Avg Deaths": avg_deaths,
        "Level": level,
        "County Count": len(counties),
        "Counties": ", ".join(counties)
    }

    node_summaries.append(summary)

# Step 3: Create DataFrame
node_summary_df = pd.DataFrame(node_summaries)

# Step 4: Save to CSV (optional)
node_summary_df.to_csv("node_summary.csv", index=False)

# You can also display it directly
node_summary_df.head()


,Node ID,Avg Deaths,Level,County Count,Counties
0,cube4_cluster0,101.0,High,1,Virginia Beach City
1,cube4_cluster1,98.0,High,1,Prince William County
2,cube5_cluster0,101.0,High,1,Virginia Beach City
3,cube5_cluster1,98.0,High,1,Prince William County
4,cube9_cluster0,107.0,High,1,Chesterfield County


In [159]:
plotly_graph_data = plotly_graph(
    kmgraph,
    graph_layout='fr',
    colorscale=my_colorscale,
    factor_size=2.5,
    edge_linewidth=0.5
)

layout = plot_layout(
    title="TDA Mapper: Overdose Clustering by County",
    width=800, height=700,
    annotation_text=get_kmgraph_meta(mapper_summary)
)

fig = go.FigureWidget(plotly_graph_data + layout)
fig.show()


TypeError: can only concatenate list (not "dict") to list

In [150]:
import numpy as np

# Build one average death count per node
color_values = []

for node_id, member_ids in scomplex['nodes'].items():
    subset_deaths = mapper_data.iloc[member_ids]['Overdose Death Count']
    avg_death = subset_deaths.mean()
    color_values.append(avg_death)


In [151]:
scomplex['nodes'].items()

dict_items([('cube0_cluster0', [6056, 6058, 6059, 6067, 6073, 6074, 6076, 6083, 6092, 6093, 6094, 6102, 6103, 6110, 6114, 6115, 16248, 16250, 16251, 16259, 16265, 16266, 16268, 16275, 16284, 16285, 16286, 16294, 16295, 16302, 16306, 16307]), ('cube0_cluster1', [6050, 6051, 6054, 6060, 6069, 6070, 6077, 6080, 6082, 6088, 6097, 6100, 6109, 6116, 6117, 16242, 16243, 16246, 16252, 16261, 16262, 16269, 16272, 16274, 16280, 16289, 16292, 16301, 16308, 16309]), ('cube1_cluster0', [6046, 6050, 6051, 6054, 6060, 6069, 6077, 6080, 6086, 6088, 6097, 6100, 6109, 6116, 6117, 16238, 16242, 16243, 16246, 16252, 16261, 16269, 16272, 16278, 16280, 16289, 16292, 16301, 16308, 16309]), ('cube1_cluster1', [6063, 6066, 6079, 6087, 6096, 6105, 6119, 16255, 16258, 16271, 16279, 16288, 16297, 16311]), ('cube2_cluster0', [6063, 6066, 6079, 6087, 6096, 6105, 6119, 16255, 16258, 16271, 16279, 16288, 16297, 16311]), ('cube2_cluster1', [6072, 6085, 16264, 16277]), ('cube3_cluster0', [6047, 6061, 6068, 6078, 6085, 

In [147]:
color_values = np.array(color_values)


In [148]:
my_colorscale = 'plasma'  # or 'pl_brewer', 'viridis', etc.

kmgraph, mapper_summary, colorf_distribution = get_mapper_graph(
    scomplex,
    color_values=color_values,
    color_function_name="Average Overdose Deaths",
    colorscale=my_colorscale
)


IndexError: index 6056 is out of bounds for axis 0 with size 41

In [ ]:
#color_values = lens [:,0] - lens[:,0].min()
#my_colorscale = pl_brewer
#kmgraph,  mapper_summary, colorf_distribution = get_mapper_graph(scomplex,
#                                                                 color_values,
#                                                                 color_function_name='Distance to x-min',
#                                                                 colorscale=my_colorscale)

# assign to node['custom_tooltips']  the node label (0 for benign, 1 for malignant)
#for node in kmgraph['nodes']:
#    node['custom_tooltips'] = y[scomplex['nodes'][node['name']]]

In [138]:
bgcolor = 'rgba(10,10,10, 0.9)'
y_gridcolor = 'rgb(150,150,150)'# on a black background the gridlines are set on  grey


In [139]:
plotly_graph_data = plotly_graph(kmgraph, graph_layout='fr', colorscale=my_colorscale,
                                 factor_size=2.5, edge_linewidth=0.5)
layout = plot_layout(title='Topological network representing the<br> Drug Overdoses in Virginia',
                     width=620, height=570,
                     annotation_text=get_kmgraph_meta(mapper_summary),
                     bgcolor=bgcolor)

fw_graph = go.FigureWidget(data=plotly_graph_data, layout=layout)
fw_hist = node_hist_fig(colorf_distribution, bgcolor=bgcolor,
                        y_gridcolor=y_gridcolor)
fw_summary = summary_fig(mapper_summary, height=300)
dashboard = hovering_widgets(kmgraph,
                             fw_graph,
                             ctooltips=True, # ctooltips = True, because we assigned a label to each
                                             #cluster member
                             bgcolor=bgcolor,
                             y_gridcolor=y_gridcolor,
                             member_textbox_width=600)

#Update the fw_graph colorbar, setting its title:

fw_graph.data[1].marker.colorbar.title = 'dist to<br>x-min'


In [140]:
VBox([fw_graph, HBox([fw_summary, fw_hist])])

    'data': [{'hoverinfo': 'none',
              'line': {'color': 'rgb(180,180,…

In [141]:
# Save the interactive graph as HTML
fw_graph.write_html("mapper_interactive_graph_KMeans.html")


In [126]:
import webbrowser
webbrowser.open("mapper_interactive_graph.html")


True

In [123]:
from IPython.display import display

# Display just the interactive graph:
display(fw_graph)

# Display the full dashboard with hover info and summaries:
display(dashboard)


FigureWidget({
    'data': [{'hoverinfo': 'none',
              'line': {'color': 'rgb(180,180,180)', 'width': 0.5},
              'mode': 'lines',
              'type': 'scatter',
              'uid': '054a4110-1c81-4726-9cdf-26aa2db1cb72',
              'x': [4.755806185145018, 5.18221003579644, None, 5.18221003579644,
                    5.549440668512609, None, 2.694883229280568, 3.4549019045252067,
                    None, 5.549440668512609, 5.78333749531536, None,
                    0.4429468759785177, -0.31086704429339757, None,
                    0.4429468759785177, 0.06768986543969437, None,
                    0.4429468759785177, 0.5995111471265525, None,
                    4.475115697712482, 3.2352910989355195, None,
                    -0.31086704429339757, 0.06768986543969437, None,
                    -0.31086704429339757, 0.5995111471265525, None,
                    -2.8343983691579133, -3.0867907176464424, None,
                    -2.8343983691579133, -1.743995550

    'data': [{'hoverinfo': 'none',
              'line': {'color'…

In [106]:
node_ids = list(graph['nodes'].keys())

#color_function_dict = {nid: node_avg_deaths[i] for i, nid in enumerate(node_ids)}
#tooltips_dict = {nid: node_tooltips[i] for i, nid in enumerate(node_ids)}

color_function_arr = np.array([color_function_dict[nid] for nid in node_ids])
tooltips_arr = [tooltips_dict[nid] for nid in node_ids]

mapper.visualize(
    graph,
    path_html="mapper_overdose.html",
    title="TDA Mapper: Overdose Patterns by County and Year",
    custom_tooltips=tooltips_arr,
    color_function=color_function_arr,
    color_function_name="Average Deaths"
)


IndexError: index 45 is out of bounds for axis 0 with size 38

In [ ]:

# KeplerMapper doesn't support dicts directly — so remap to list in same order
node_ids = list(graph['nodes'].keys())
color_function_arr = np.array([color_function_dict[nid] for nid in node_ids])
tooltips_arr = [tooltips_dict[nid] for nid in node_ids]

# Now visualize
mapper.visualize(
    graph,
    path_html="mapper_overdose.html",
    title="TDA Mapper: Overdose Patterns by County and Year",
    custom_tooltips=tooltips_arr,
    color_function=color_function_arr,
    color_function_name="Average Deaths"
)


KeyError: 0

In [83]:
print("Node IDs:", list(graph["nodes"].keys()))
print("Max node ID:", max(graph["nodes"].keys()))
print("Number of nodes:", len(graph["nodes"]))


Node IDs: ['cube0_cluster0', 'cube0_cluster1', 'cube1_cluster0', 'cube1_cluster1', 'cube2_cluster0', 'cube2_cluster1', 'cube2_cluster2', 'cube2_cluster3', 'cube2_cluster4', 'cube2_cluster5', 'cube3_cluster0', 'cube3_cluster1', 'cube3_cluster2', 'cube3_cluster3', 'cube4_cluster0', 'cube4_cluster1', 'cube4_cluster2', 'cube4_cluster3', 'cube5_cluster0', 'cube5_cluster1', 'cube5_cluster2', 'cube5_cluster3', 'cube6_cluster0', 'cube6_cluster1', 'cube6_cluster2', 'cube6_cluster3', 'cube7_cluster0', 'cube7_cluster1', 'cube7_cluster2', 'cube7_cluster3', 'cube7_cluster4', 'cube7_cluster5', 'cube7_cluster6', 'cube8_cluster0', 'cube8_cluster1', 'cube8_cluster2', 'cube9_cluster0', 'cube9_cluster1']
Max node ID: cube9_cluster1
Number of nodes: 38


In [84]:
node_ids = list(graph['nodes'].keys())
print("Length of node_avg_deaths:", len(node_avg_deaths))
print("Length of node_ids:", len(node_ids))
print("Sample node ID:", node_ids[0])
print("Sample tooltip:", node_tooltips[node_ids[0]])
print("Sample avg deaths:", node_avg_deaths[node_ids[0]])


Length of node_avg_deaths: 38
Length of node_ids: 38
Sample node ID: cube0_cluster0
Sample tooltip: <b>Node cube0_cluster0</b><br>Most Common County: Accomack County<br>Total Deaths: 13820.0<br>Avg Deaths: 4.72<br>Std Dev Deaths: 10.52<br>Year Range: 2019 – 2019<br>Points: 2926
Sample avg deaths: 4.723171565276829


In [81]:
# Sort node IDs
node_ids = list(graph['nodes'].keys())

# Align values with node order
avg_deaths_ordered = [node_avg_deaths[nid] for nid in node_ids]
tooltips_ordered = [node_tooltips[nid] for nid in node_ids]

mapper.visualize(
    graph,
    path_html="mapper_overdose.html",
    title="TDA Mapper: Overdose Patterns by County and Year",
    custom_tooltips=tooltips_ordered,
    color_function=np.array(avg_deaths_ordered),
    color_function_name="Average Deaths"
)


IndexError: index 45 is out of bounds for axis 0 with size 38

In [ ]:
node_tooltips = []
node_avg_deaths = []

for node_id, indices in graph['nodes'].items():
    subset = mapper_data.iloc[indices]

    deaths = subset['Overdose Death Count']
    counties = subset['County']
    years = subset['Overdose Death Year']

    tooltip = (
        f"<b>Node {node_id}</b><br>"
        f"Most Common County: {counties.mode().iloc[0]}<br>"
        f"Total Deaths: {deaths.sum()}<br>"
        f"Avg Deaths: {deaths.mean():.2f}<br>"
        f"Std Dev Deaths: {deaths.std():.2f}<br>"
        f"Year Range: {years.min()} – {years.max()}<br>"
        f"Points: {len(indices)}"
    )

    node_tooltips.append(tooltip)
    node_avg_deaths.append(deaths.mean())  # For coloring

# Visualize with color function and custom tooltips
mapper.visualize(
    graph,
    path_html="mapper_overdose.html",
    title="TDA Mapper: Overdose Patterns by County and Year",
    custom_tooltips=node_tooltips,
    color_function=np.array(node_avg_deaths),
    color_function_name="Average Deaths"

)


0

In [ ]:

# Assuming df is your DataFrame
data_standardized = (mapper_data - mapper_data.mean()) / mapper_data.std()
